In [1]:
import jax
import jax.numpy as np
jax.config.update('jax_enable_x64', True)
import matplotlib.pyplot as plt

import sys
sys.path.append('../')
import schrodinger

## Introduction

This repository is built to simulate the Schrödinger equation. Provided a potential $V(t, x)$ and moving boundaries $[x_1(t),x_2(t)]$, a solution wavefunction $\psi$ satisfies

$$
i\partial_t \psi(t, x) = \left(-\frac{1}{2}\partial_x^2+V(t, x)\right)\psi(t, x)
$$

and it can be made to satisfy periodic, Dirichlet or absorbing (PML) boundary conditions at $[x_1(t),x_2(t)]$.

To build the problem we'll define functions for $x_1(t),x_2(t),V(t, x)$ and feed them to our numerical solver for later analysis and visualisation. This will return a `Wavefunction` object which we'll look at briefly. It contains information about a wavefunction and its energy.

Let's define some example data, bearing in mind that a `Wavefunction` normalises its data at each time step &mdash; you need not input a normalised amplitude.

In [2]:
Tmax = 2
L = 1
k = 2 * np.pi / L
w = k**2 / 2

t = np.linspace(0, Tmax, 200)
x = np.linspace(-L/2, L/2, 300)
x1 = lambda t: -L/2
x2 = lambda t: L/2 
V  = lambda t, x: 0
T, X = np.meshgrid(t, x, indexing='ij')
amplitude = np.cos(k * X) * np.exp(-1j * w * T)
psi = schrodinger.Wavefunction(amplitude, t, x1, x2, V, BC='periodic')

The `Wavefunction`object has two methods avaliable:
 - `plot`, which shows a 3D view of the wavefunction. Optional arguments for `t` and `x` show 2D plots of the wavefunction at different instants of time or space.
 - `animate`, which returns an animation of the wavefunction.

In [3]:
psi.plot()
plt.show()

In [4]:
psi.plot(t=1)
plt.show()

In [5]:
psi.plot(x=.2)
plt.show()

In [6]:
ani = psi.animate()
plt.show()

With the basics covered, let's check out the eigenfunctions of well known systems: the particle in a box and the harmonic oscillator. We'll do this using the `solve` function, which requires:
- Initial wavefunction
- Times array
- Left boundary 
- Right boundary
- Potential
- Boundary conditions (`periodic`/`dirichlet`/`pml`)

In [7]:
# Particle in a box

L = 1
n = 3
k = n * np.pi / L
w = k**2 / 2
T = 4 / w

def PIB(n, x):
    L = x[-1] - x[0]
    k = n * np.pi / L 
    if n%2:
        return np.cos(k*x)
    else:
        return np.sin(k*x)

t = np.linspace(0, T, 100)
x = np.linspace(-L/2, L/2, 1000)
psi0 = PIB(n, x)
x1 = lambda t: -L/2
x2 = lambda t: L/2 
V  = lambda t, x: 0
psi = schrodinger.solve(psi0, t, x1, x2, V, BC='dirichlet')

In [8]:
psi.plot(t=0)
plt.show()

In [9]:
psi.plot(x=0)
plt.show()

In [10]:
psi.plot()
plt.show()

In [11]:
ani = psi.animate()
plt.show()

In [21]:
# Harmonic oscillator
from scipy.special import hermite

w = 1
n = 3
L = 6 * np.sqrt((2*n+1)/w)
T = 4 / w

def HO(n, x):
    Hn = hermite(n)
    return np.exp(-w/2*x**2) * Hn(np.sqrt(w)*x)

t = np.linspace(0, T, 100)
x = np.linspace(-L/2, L/2, 1000)
psi0 = HO(n, x)
x1 = lambda t: -L/2
x2 = lambda t: L/2 
V  = lambda t, x: w**2/2 * x**2
psi = schrodinger.solve(psi0, t, x1, x2, V, BC='periodic')

In [13]:
psi.plot()
plt.show()

In [24]:
ani = psi.animate()
plt.show()

Now that we have covered some basic examples, why not play around with moving boundaries and arbitrary initial conditions?

In [29]:
L = 1
n = 3
k = n * np.pi / L
w = k**2 / 2
T = 30 / w

t = np.linspace(0, T, 200)
x = np.linspace(-L/2, L/2, 1000)
psi0 = PIB(n, x)
x1 = lambda t: -L/2 - .01 * w * t
x2 = lambda t: L/2  + .01 * w * t
V  = lambda t, x: 0
psi = schrodinger.solve(psi0, t, x1, x2, V, BC='dirichlet')

In [30]:
ani = psi.animate()
plt.show()

In [17]:
L = 1
T = 2

t = np.linspace(0, T, 100)
x = np.linspace(-L/2, L/2, 1000)
psi0 = 5* (x-L/2)*(x+L/2)**3
x1 = lambda t: -1/2 - .05 * t
x2 = lambda t: 1/2 + .05  * t 
V  = lambda t, x: 0
psi = schrodinger.solve(psi0, t, x1, x2, V, BC='dirichlet')

In [18]:
ani = psi.animate()
plt.show()

In [19]:
L = 10
T = 2
k0 = 100/L
x0 = -L/4
s = L/10

t = np.linspace(0, T, 100)
x = np.linspace(-L/2, L/2, 1000)
psi0 = np.exp(1j*k0*x)*np.exp(-(x-x0)**2/(2*s**2))
V  = lambda t, x: 0
x1 = lambda t: -L/2
x2 = lambda t: L/2
psi = schrodinger.solve(psi0, t, x1, x2, V, BC='pml')

In [20]:
ani = psi.animate()
plt.show()